In [1]:
import time
import json
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC

In [2]:
options = Options()
options.add_argument("--incognito")
options.add_argument("--disable-notifications")
options.add_argument("user-agent={}".format(UserAgent(use_cache_server = False).random))

# Open Chrome Browser
chrome = webdriver.Chrome('./chromedriver', chrome_options = options)
chrome.get("https://www.google.com/maps")
chrome.maximize_window()

C:\Users\dw1209\AppData\Local\Temp/ipykernel_10528/1986086670.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chrome = webdriver.Chrome('./chromedriver', chrome_options = options)
C:\Users\dw1209\AppData\Local\Temp/ipykernel_10528/1986086670.py:7: DeprecationWarning: use options instead of chrome_options
  chrome = webdriver.Chrome('./chromedriver', chrome_options = options)


In [3]:
# Filled Restaurant in the search box and click the search button
search_box = chrome.find_element(By.ID, "searchboxinput")
search_box.send_keys("超商")
chrome.find_element(By.XPATH, "/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/div[1]/button").click()

# Wait for it
time.sleep(3)

In [4]:
titles, comment_counts, urls = list(), list(), list()

In [5]:
# 400 times
for number in range(20):
    for count in range(20):
        pane = chrome.find_element(By.XPATH, '/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]')
        
        # Mouse Scroll Down
        for wait_count in range(3):    
            chrome.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)
            time.sleep(1)
        
        # Wait for it
        time.sleep(1)
        
        try:
            # Click the store we would like to fetch the datas
            # Hospital, Con: div.Nv2PK.tH5CWc.THOPZb, Others: div.Nv2PK.THOPZb.CpccDe
            results = chrome.find_elements(By.CSS_SELECTOR, 'div.Nv2PK.tH5CWc.THOPZb')
            results[count].click()
            time.sleep(2)

            soup = BeautifulSoup(chrome.page_source, "lxml")
            
            # Url
            try: urls.append(chrome.current_url)
            except: urls.append(None)
            
            # Title
            try: titles.append(soup.find_all(class_ = 'DUwDvf fontHeadlineLarge')[0].text)
            except: titles.append(None)
            
            # Comments
            try: comment_counts.append(soup.find_all(class_ = 'F7nice mmu3tf')[1].text)
            except: comment_counts.append(None)

            # Return back
            chrome.find_element(By.XPATH, "/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[1]/button").click()
            time.sleep(2)
        except:
            # Mouse Scroll Up
            # chrome.execute_script("arguments[0].scrollTop = 0", pane)
            # time.sleep(1)
            continue
        
    # Click the next page
    chrome.find_element(By.XPATH, '/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div/button[2]').click()
    time.sleep(3)

ElementClickInterceptedException: Message: element click intercepted: Element <button disabled="true" ved="1i:2,t:12696,e:1,p:HBGAYrSWKeSZr7wP-I-LkAY:70952" aria-label="下一頁" id="ppdPk-Ej1Yeb-LgbsSe-tJiF1e" jstcache="876" jsaction="pane.paginationSection.nextPage" class="hV1iCc Hk4XGb I6WOUe" jsan="7.hV1iCc,7.Hk4XGb,7.I6WOUe,0.disabled,0.ved,0.aria-label,0.id,22.jsaction">...</button> is not clickable at point (367, 858). Other element would receive the click: <div class="punXpd">...</div>
  (Session info: chrome=101.0.4951.64)
Stacktrace:
Backtrace:
	Ordinal0 [0x008FB8F3+2406643]
	Ordinal0 [0x0088AF31+1945393]
	Ordinal0 [0x0077C748+837448]
	Ordinal0 [0x007AFA1B+1047067]
	Ordinal0 [0x007AD9F8+1038840]
	Ordinal0 [0x007AB6DB+1029851]
	Ordinal0 [0x007AA439+1025081]
	Ordinal0 [0x007A0083+983171]
	Ordinal0 [0x007C427C+1131132]
	Ordinal0 [0x0079FA64+981604]
	Ordinal0 [0x007C4494+1131668]
	Ordinal0 [0x007D4682+1197698]
	Ordinal0 [0x007C4096+1130646]
	Ordinal0 [0x0079E636+976438]
	Ordinal0 [0x0

In [6]:
google_map_df = pd.DataFrame({
    "Title": titles,
    "Comments": comment_counts,
    "Url": urls,
})

In [7]:
google_map_df

,Title,Comments,Url
0,全家便利商店 新竹交大店,36 則評論,https://www.google.com/maps/place/%E5%85%A8%E5...
1,7-ELEVEN 科技城門市,112 則評論,https://www.google.com/maps/place/7-ELEVEN+%E7...
2,7-ELEVEN 清大門市,82 則評論,https://www.google.com/maps/place/7-ELEVEN+%E6...
3,7-ELEVEN 科有門市,37 則評論,https://www.google.com/maps/place/7-ELEVEN+%E7...
4,7-ELEVEN GRC門市,4 則評論,https://www.google.com/maps/place/7-ELEVEN+GRC...
...,...,...,...
216,全家便利超商,75 則評論,https://www.google.com/maps/place/%E5%85%A8%E5...
217,7-ELEVEN 航海門市,34 則評論,https://www.google.com/maps/place/7-ELEVEN+%E8...
218,OK便利商店 觀音中興店,14 則評論,https://www.google.com/maps/place/OK%E4%BE%BF%...
219,全家便利商店 桃園鼎聖店,28 則評論,https://www.google.com/maps/place/%E5%85%A8%E5...


In [10]:
# google_map_df.to_csv("attraction.csv", encoding = "utf-16", index = False)
# google_map_df.to_csv("restaurant.csv", encoding = "utf-16", index = False)
# google_map_df.to_csv("bookstore.csv", encoding = "utf-16", index = False)
# google_map_df.to_csv("department.csv", encoding = "utf-16", index = False)
# google_map_df.to_csv("nightmarket.csv", encoding = "utf-16", index = False)
# google_map_df.to_csv("outlet.csv", encoding = "utf-16", index = False)
# google_map_df.to_csv("clinic.csv", encoding = "utf-16", index = False)
# google_map_df.to_csv("hospital.csv", encoding = "utf-16", index = False)
# google_map_df.to_csv("convenience.csv", encoding = "utf-16", index = False)